In [1]:
import numpy as np
import cv2
import os

path = os.getcwd()
DATA_PATH = os.path.join(path, "data")
IMG_PATH = os.path.join(DATA_PATH, "dataset", "train")
model_path = os.path.join(DATA_PATH, "pre-trained")

caffe_model_path = os.path.join(model_path, os.listdir(model_path)[0])
deploy_proto_path = os.path.join(model_path, os.listdir(model_path)[1])
_confidence = 0.30

print(caffe_model_path)
print(deploy_proto_path)
print(_confidence)


CLASSES = ["background", "aeroplane", "bicycle", "bird", "boat",
	"bottle", "bus", "car", "cat", "chair", "cow", "diningtable",
	"dog", "horse", "motorbike", "person", "pottedplant", "sheep",
	"sofa", "train", "tvmonitor"]
COLORS = np.random.uniform(0, 255, size=(len(CLASSES), 3))

print("[INFO] loading model...")
net = cv2.dnn.readNetFromCaffe(deploy_proto_path, caffe_model_path)

for im in os.listdir(IMG_PATH):
    image = cv2.imread(os.path.join(IMG_PATH, im))
    # cv2.imshow("DD", image)
    (h, w) = image.shape[:2]
    blob = cv2.dnn.blobFromImage(cv2.resize(image, (300, 300)), 0.007843, (300, 300), 127.5)

    # pass the blob through the network and obtain the detections and
    # predictions
    print("[INFO] computing object detections...")
    net.setInput(blob)
    detections = net.forward()


    # loop over the detections
    for i in np.arange(0, detections.shape[2]):
        # extract the confidence (i.e., probability) associated with the
        # prediction
        confidence = detections[0, 0, i, 2]
        # filter out weak detections by ensuring the `confidence` is
        # greater than the minimum confidence
        if confidence > _confidence:
            # extract the index of the class label from the `detections`,
            # then compute the (x, y)-coordinates of the bounding box for
            # the object
            idx = int(detections[0, 0, i, 1])
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")
            # display the prediction
            label = "{}: {:.2f}%".format(CLASSES[idx], confidence * 100)
            print("[INFO] {}".format(label))
            cv2.rectangle(image, (startX, startY), (endX, endY),
                COLORS[idx], 2)
            y = startY - 15 if startY - 15 > 15 else startY + 15
            cv2.putText(image, label, (startX, y),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, COLORS[idx], 2)

    # show the output image

    cv2.imshow("Output", image)
    if cv2.waitKey(0) & 0xFF == ord('q'):
        break


/home/asifhaider/Dev/ML/dnn-object-detection/data/pre-trained/mobilenet_iter_73000.caffemodel
/home/asifhaider/Dev/ML/dnn-object-detection/data/pre-trained/deploy.prototxt
0.3
[INFO] loading model...
[INFO] computing object detections...
[INFO] person: 79.38%
[INFO] person: 45.88%
[INFO] computing object detections...
[INFO] bicycle: 97.86%
[INFO] motorbike: 49.46%
[INFO] person: 43.59%
[INFO] computing object detections...
[INFO] bicycle: 91.23%
[INFO] person: 37.16%
[INFO] computing object detections...
[INFO] car: 99.61%
